# TF-IDF utilizando Gensim

## Importar librerias

In [29]:
import gensim
import pprint
from gensim import corpora
from gensim.utils import simple_preprocess

## Lectura de los archivos

En este punto, la idea es tener en un dataframe de pandas tanto el nombre del documento (nombre de archivo), como su contenido

In [15]:
doc_list = [
   "Hello, how are you?", "How do you do?", 
   "Hey what are you doing? yes you What are you doing?"
]

doc_list
df = pd.DataFrame({"doc": [1,2,3],"content": doc_list})
df.head()

,doc,content
0,1,"Hello, how are you?"
1,2,How do you do?
2,3,Hey what are you doing? yes you What are you d...


## Preparación de textos

Para comenzar cualquier tarea de NLP (clasificación, recomendación, etc.) se debe preparar cada documento del *corpus*.

Normalmente esto incluye tareas como:

- Tokenización
- Quitar *stop words*
- Quitar caracteres especiales.
- Lematización y/o *Stemming* (¿Cúal es mejor?)
- Creación de un diccionario
- Creación de una bolsa de palabras (*BoW*)

### Tokenización

In [20]:
df["tokens"] = df.content.apply(lambda doc: simple_preprocess(doc))

En este punto, se pueden incluir mas pasos del preprocesing utilizando el `df[".."].apply(lambda x: ...)`

### Creación del diccionario

In [23]:

dictionary = corpora.Dictionary(df["tokens"])

El objeto **dictionary** es un arreglo de palabras únicas de todo el corpus (todos los documentos)

`['are', 'hello', 'how', 'you', 'do', 'doing', 'hey', 'what', 'yes']`

### Creación del BoW

In [50]:
raw_corpus = [dictionary.doc2bow(doc) for doc in df["tokens"]]
raw_corpus

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(2, 1), (3, 1), (4, 2)],
 [(0, 2), (3, 3), (5, 2), (6, 1), (7, 2), (8, 1)]]

*raw_corpus* es el famoso **Bag of Words**. El BoW es una matriz, donde cada fila representa un documento, y cada columna tiene una tupla, la cual tiene el formato *(id,count)*. Este *id* es la identificación (o posición) de cada palabra del documento dentro del diccionario.

Para entender un poco más lo que nos dice el **BoW**, podemos utilizar el **dictionary** para extraer el token a partir de un id.

In [56]:
for doc in raw_corpus:
   print([[dictionary[id], freq] for id, freq in doc])

[['are', 1], ['hello', 1], ['how', 1], ['you', 1]]
[['how', 1], ['you', 1], ['do', 2]]
[['are', 2], ['you', 3], ['doing', 2], ['hey', 1], ['what', 2], ['yes', 1]]


## TF-IDF

**Term Frecuency-Inverted Document Frecuency** tiene dos partes: 
- *Term Frecuency*: número de veces que esta una palabra en cada documento
- *Inverted Document Frecuency*: Que tan rara es una palabra dentro del *corpus*

Esta metrica nos ayuda a identificar esas palabras son relevantes para un documento, porque se repite varias veces dentro de él, pero es escasa entre los documentos.

**NOTA:** no confundir con palabras con los topicos de tecnícas como LSA o LDA

In [53]:
tfidf = gensim.models.TfidfModel(raw_corpus)

La primera parte de la tarea de **TF-IDF** ya fue realizada cuando creamos el *BoW*, lo que hace `TfidfModel` es la parte de IDF, el cual requiere que el `corpus` este dividio en diferentes *documentos*. Si todos los documentos son unidos en un gran *string*, esta fase de IDF no tendría sentido.

In [39]:
import numpy as np
for doc in tfidf[raw_corpus]:
   print([[dictionary[id], np.round(freq,2)] for id, freq in doc])

[['are', 0.33], ['hello', 0.89], ['how', 0.33]]
[['how', 0.18], ['do', 0.98]]
[['are', 0.23], ['doing', 0.62], ['hey', 0.31], ['what', 0.62], ['yes', 0.31]]
